In [ ]:
from lsst.daf.persistence import Butler

butler = Butler('/project/tmorton/tickets/DM-20015/RC2_w18')
# butler = Butler('/project/tmorton/tickets/DM-19684')

In [ ]:
tract = 9615
filt = 'HSC-I'
coaddParq = butler.get('analysisCoaddTable_forced', tract=tract, filter=filt)
visitParq = butler.get('analysisVisitTable', visit=1258, tract=tract, filter=filt)

In [ ]:
metric_columns = ['base_Footprint_nPix', 'Gaussian-PSF_magDiff_mmag',
                  'CircAper12pix-PSF_magDiff_mmag', 'Kron-PSF_magDiff_mmag',
                  'CModel-PSF_magDiff_mmag', 'traceSdss_pixel',
                  'traceSdss_fwhm_pixel', 'psfTraceSdssDiff_percent',
                  'e1ResidsSdss_milli', 'e2ResidsSdss_milli',
                  'deconvMoments', 'compareUnforced_Gaussian_magDiff_mmag',
                  'compareUnforced_CircAper12pix_magDiff_mmag',
                  'compareUnforced_Kron_magDiff_mmag',
                  'compareUnforced_CModel_magDiff_mmag']

flag_columns = ['deblend_nChild',
                'calib_psf_used', 'calib_psf_candidate', 'calib_photometry_reserved',
                'merge_measurement_i2', 'merge_measurement_i', 'merge_measurement_r2',
                'merge_measurement_r', 'merge_measurement_z', 'merge_measurement_y',
                'merge_measurement_g', 'merge_measurement_N921', 'merge_measurement_N816',
                'merge_measurement_N1010', 'merge_measurement_N387', 'merge_measurement_N515',
                'qaBad_flag']

In [ ]:
# These are the metrics/flags that *are* available in the visit tables

for c in metric_columns + flag_columns:
    if c in visitParq.columns:
        print(c)

In [ ]:
# These are the metrics/flags that are *unavailable* in the visit tables

for c in metric_columns + flag_columns:
    if c not in visitParq.columns:
        print(c)

In [ ]:
visitMatchParq = butler.get('visitMatchTable', tract=tract, filter=filt)
match_df = visitMatchParq.toDataFrame()

In [ ]:
# Select just the "psf_used" IDs to use as a demo subset

from lsst.qa.explorer.functors import Column, Index

psfUsed = Column('calib_psf_used')(coaddParq)
psfUsedIds = Index()(coaddParq)[psfUsed]
visit_psfUsedIds = match_df.loc[psfUsedIds]['matchId']

In [ ]:
visit_psfUsedIds.head()

In [ ]:
all_visits = list(visit_psfUsedIds.columns)
all_visits.sort()

all_visitParqs = {v: butler.get('analysisVisitTable', visit=v, tract=tract, filter=filt) for v in all_visits}

In [ ]:
import pandas as pd

def get_allvisits_df(visits, coaddIds, column):
    """Returns a df of all values from a particular column in all visits, with each column being a visit
    """
    df = pd.DataFrame(index=coaddIds, columns=visits)
    for v in visits:
        s = all_visitParqs[v].toDataFrame(columns=[column])
        ids = visit_psfUsedIds[v]
        ok = ids.notnull()
        df.loc[coaddIds[ok], v] = s.loc[ids[ok]].values.squeeze()  # don't know why this has weird dimensions    
    return df

In [ ]:
gausspsf_df = get_allvisits_df(all_visits, psfUsedIds, 'Gaussian-PSF_magDiff_mmag')

In [ ]:
gausspsf_df

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
%%opts Curve [width=800, height=300, tools=['hover']] 

mean_df = gausspsf_df.mean()
std_df = gausspsf_df.std()

mean_points = [(str(v), mean_df.loc[v]) for v in all_visits]
std_points = [(str(v), std_df.loc[v]) for v in all_visits]

hv.Curve(mean_points, label='mean (gauss-psf)') * hv.Curve(std_points, label='std (gauss-psf)')
